# TORCH.AUTOGRAD를 사용한 자동 미분
- 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 **역전파**이다.
- 이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정된다.

- 이러한 변화도를 계산하기 위해, Pytorch에서 `torch.autograd` 라고 불리는 자동 미분 엔진이 내장되어 있다.
- 이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원한다.

- 입력 x, 매개변수 `w`와 `b`, 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 만들어보자.

In [1]:
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad= True)
b = torch.randn(3, requires_grad= True)
z = torch.matmul(x,w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

c:\Users\admin\anaconda3\envs\k_public\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
